<a target="_blank" href="https://colab.research.google.com/github/okareo-ai/okareo-python-sdk/blob/main/examples/classification_eval_with_OpenAI.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Evaluate classification with OpenAI:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the API settings by clicking the button under **"1. Create API Token"**. You can skip all other steps.

4. Add your generated API token to the cell below. 👇

5. Add your OpenAI key.  OpenAI is only needed for example purposes.  If you have your own model, you can substitute those in it's place.

In [1]:
OKAREO_API_KEY = "<YOUR-OKAREO-API-KEY>"
OPENAI_API_KEY = "<YOUR-OPENAI-API-KEY>"

In [ ]:
%pip install okareo
%pip install openai

In [2]:
# Simple adhoc classifier using OpenAI'a GPT 3.5 Turbo model

USER_PROMPT_TEMPLATE = "{input}"

CLASSIFICATION_CONTEXT_TEMPLATE = """
You will be provided a question from a customer.
Classify the question into a customer category and sub-category.
Provide the output with only the category name.

Categories: Technical Support, Billing, Account Management, General Inquiry, Unknown

Sub-Categories for Technical Support:
Troubleshooting
Product features
Product updates
Integration options
Found a problem

Sub-Categories for Billing:
Unsubscribe
Upgrade
Explain my bill
Change payment
Dispute a charge

Sub-Categories for Account Management:
Add a team member
Change or Update details
Password reset
Close account
Security

Sub-Categories for General Inquiry:
Contact sales
Product information
Pricing
Feedback
Speak to a human
"""

In [3]:
# EXAMPLE calling the OpenAI classifier directly to play with how it responds to different inputs

from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

def get_turbo_classification(messages, model="gpt-3.5-turbo", 
  temperature=0, max_tokens=500):
  response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, 
    max_tokens=max_tokens,
  )
  return response.choices[0].message.content

user_message = "I can't connect my CRM"
user_message_unsub= "Please unsubscribe me from your service"
user_message_crazy= "My cat is on fire."
messages =  [  
    {'role':'system', 'content': CLASSIFICATION_CONTEXT_TEMPLATE},    
    {'role':'user', 'content': user_message_unsub}
]

class_response = get_turbo_classification(messages)
print(class_response)

Billing


In [5]:
# Run the evaluation on the model and get a link to the results in Okareo
import random
import string
from okareo import Okareo
from okareo.model_under_test import OpenAIModel
from okareo_api_client.models.test_run_type import TestRunType
from okareo_api_client.models import ScenarioSetCreate, SeedData, ScenarioType

okareo = Okareo(OKAREO_API_KEY)
random_string = ''.join(random.choices(string.ascii_letters, k=5))

# Establish the model that is being evaluated, at minimum this is a named model for future reference
model_under_test = okareo.register_model(
        # name=f"AdHoc OpenAI Classifier - {random_string}",
        name=f"AdHoc OpenAI Classifier - CHANGE TEST",
        model=OpenAIModel(
            model_id="gpt-3.5-turbo",
            temperature=0,
            system_prompt_template=CLASSIFICATION_CONTEXT_TEMPLATE,
            user_prompt_template=USER_PROMPT_TEMPLATE,
        ),
    )

# Generate example scenario based on seed data
random_string = ''.join(random.choices(string.ascii_letters, k=5))
scenario_set_create = ScenarioSetCreate(
    name=f"OpenAI AdHoc Category - {random_string}",
    seed_data=[
        SeedData(
            input_="Can I connect my CRM?",  
            result="Technical Support"
        ),
        SeedData(
            input_="Do you have a way to send marketing emails?",  
            result="Technical Support"
        ),
        SeedData(
            input_="Can I get invoiced instead of using a credit card?", 
            result="Billing"
        ),
        SeedData(
            input_="My CRM integration is not working.", 
            result="Technical Support"
        ),
        SeedData(
            input_="Do you have SOC II tpye 2 certification?", 
            result="Account Management"
        ),
        SeedData(
            input_="I like the product.  Please connect me to your enterprise team.", 
            result="General Inquiry"
        )
    ],
)
scenario = okareo.create_scenario_set(scenario_set_create)

# run the test and call the model for each item in the scenario set
evaluation = model_under_test.run_test(
        name=f"adhoc-class-run-{random_string}",
        scenario=scenario,
        api_key=OPENAI_API_KEY,
        test_run_type=TestRunType.MULTI_CLASS_CLASSIFICATION,
        calculate_metrics=True,
    )

print(evaluation.app_link)

https://app.okareo.com/project/9255bce6-8704-42f7-8e8d-867f13cd9328/eval/30568c66-faf4-4ceb-902a-9ab04912a790
